In [ ]:
import imp
import datetime
import orbitfit.tle_utils as tu
from orbitfit.io import parse_gps_data
from orbitfit.rotate import rotate_gps
from orbitfit.utils import filter_with_mean_oe, calculate_gps_tle_error

import orbitfit.orbitfit as of
import matplotlib.pyplot as plt
import numpy as np

import logging
logging.basicConfig(level=logging.INFO)

%matplotlib notebook

In [ ]:
imp.reload(of)
imp.reload(tu)

In [ ]:
filename = "NSL1_GPS_Test_20210309.csv"
df_gps = parse_gps_data(filename)

In [ ]:
df_gps_teme = rotate_gps(df_gps, method="E2T")

In [ ]:
df_gps_teme_filtered = filter_with_mean_oe(df_gps_teme, thresholds={"mean_sma": 5.0, "mean_ecc": 0.001, "mean_inc": 0.017}, plot=False)

In [ ]:
filename = "example_tle.txt"
tle = tu.load_tle(filename)

In [ ]:
tle_list = [{"tle":tle, "start": df_gps_teme_filtered.index[0], "end":df_gps_teme_filtered.index[-1]}]
df_gps_tle_error = calculate_gps_tle_error(df_gps_teme_filtered, tle_list)
df_gps_tle_error[["alongtrack_error_km",  "acrosstrack_error_km", "radial_error_km"]].plot()

In [ ]:
df_gps_teme_filtered.index = df_gps_teme_filtered.index - datetime.timedelta(seconds=18) 

In [ ]:
df_gps_tle_error = calculate_gps_tle_error(df_gps_teme_filtered, tle_list)
df_gps_tle_error[["alongtrack_error_km",  "acrosstrack_error_km", "radial_error_km"]].plot()

# Run fit

In [ ]:
tle_fit = of.TLEFit(df_gps_teme_filtered, tle)

In [ ]:
error_pos_km, error_vel_km_s = tle_fit.get_state_error(tle_fit.initial_state, tle_fit.df_gps)

fig, ax = plt.subplots(2,1,sharex=True)
ax[0].plot(tle_fit.dt_min, error_pos_km)
ax[0].set_ylabel("Position Error [km]")
ax[1].plot(tle_fit.dt_min, error_vel_km_s)
ax[1].set_ylabel("Velocity Error [km/s]")

In [ ]:
tle_out = tle_fit.run_fit(max_loops=50, deltaamtchg=1e-7)#, percentchg=0.5, deltaamtchg=1e-4, epsilon=1e-11)

In [ ]:
state = tle_fit.tle2state(tle_out)
error_pos_km, error_vel_km_s = tle_fit.get_state_error(state, tle_fit.df_gps)

fig, ax = plt.subplots(2,1,sharex=True)
ax[0].plot(tle_fit.dt_min, error_pos_km)
ax[0].set_ylabel("Position Error [km]")
ax[1].plot(tle_fit.dt_min, error_vel_km_s)
ax[1].set_ylabel("Velocity Error [km/s]")